# Higher Order Functions
- Higher order functions are functions that operate on complex data types such as arrays and maps
- They allow you to pass functions as arguments (such as lambda expressions), apply transformations and return arrays or map
- They are extremely useful for manipulating arrays without exploding them

## Commonly used Higher Order Array Functions
- TRANSFORM
- FILTER
- EXISTS
- AGGREGATE

### Syntax
<function_name> (array_column, lambda_expression)

lambda_expression: element -> expression

### Creating a temp view to perform operations

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW order_items 
AS 
SELECT * FROM
VALUES
  (1, array('smartphone', 'laptop', 'monitor')),
  (2, array('tablet', 'headphones', 'smartwatch')),
  (3, array('keyboard', 'mouse'))
AS orders(order_id, items);


In [0]:
%sql
SELECT * FROM order_items;

### 1. Conver all the item names to be UPPERCASE(TRANSFORM Function)

In [0]:
%sql
SELECT order_id, transform(items, x -> UPPER(x)) AS upper_items
FROM order_items;

### 2. Filter only items that contains the string 'smart' (FILTER Function)

In [0]:
%sql
SELECT order_id, filter(items, x -> x LIKE '%smart%') AS upper_items
FROM order_items;

### 3. Check to see wheather the order includes any 'monitor' (EXISTS Function)

In [0]:
%sql
SELECT order_id, EXISTS(items, x -> x = 'monitor') AS upper_items
FROM order_items;

### Array with more than one object

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW order_items 
AS 
SELECT * FROM
VALUES
  (1, array(
    named_struct('name', 'smartphone', 'price', 699),
    named_struct('name', 'laptop', 'price', 1199),
    named_struct('name', 'monitor', 'price', 399)
  )),
  (1, array(
    named_struct('name', 'tablet', 'price', 599),
    named_struct('name', 'headphone', 'price', 199),
    named_struct('name', 'smartwatch', 'price', 399)
  )),
  (1, array(
    named_struct('name', 'keyboard', 'price', 89),
    named_struct('name', 'mouse', 'price', 59)
  ))
AS orders(order_id, items);

In [0]:
%sql
SELECT * FROM order_items;

### 1. convert all the item names to be uppercase & add 10% tax to each item (transform function)

In [0]:
%sql
SELECT order_id,
    transform(items, x -> named_struct(
                  'name', UPPER(x.name), 
                  'price', ROUND(x.price * 1.1, 2)
                  )) items_with_tax
FROM order_items;

### 2. Calculate the total order amount for each of the order (Aggregate Functions)

In [0]:
%sql
SELECT order_id, 
        aggregate(items, 0, (acc, x) -> acc + x.price) AS total_order_price
FROM order_items;

### Map Functions
A map is a collection of key-value pairs, like a dictionary

{'laptop', 1200, 'phone': 699}

### Commonly used Higher Order Map Functions
- TRANSFORM_VALUES
- TRANSFORM_KEYS
- MAP_FILTER

### Syntax

function_name (map_column, lambda_expression)

lambda expression: (key, value) -> expression)

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW order_item_prices AS
SELECT * FROM VALUES
(1, map('smartphone', 699, 'laptop', 1199, 'monitor', 399)),
(2, map('tablet', 599, 'headphones', 199, 'smartwatch', 399)),
(3, map('keyboard', 89, 'mouse', 59))
AS orders(order_id, item_prices)

In [0]:
%sql
SELECT * FROM order_item_prices;

### 1. Convert all the item names to be UPPERCASE (TRANSFORM_KEYS Function)

In [0]:
%sql
SELECT order_id,
    transform_keys(item_prices, (item, price) -> (UPPER(item), price)) AS items_upper_case
from order_item_prices;

### 2. Apply 10% TAX to item prices (TRANSFORM_VALUES Function)

In [0]:
%sql
SELECT order_id,
    transform_values(item_prices, (item, price) -> (item, ROUND(price * 1.10, 2))) AS price_with_tax
from order_item_prices;

### 3. Filter only items with price above $500 (MAP_FILTER Function)

In [0]:
%sql
SELECT order_id,
    map_filter(item_prices, (item, price) -> price > 500) AS premium_items
from order_item_prices;